In [23]:
import pandas as pd
import json
import re

In [24]:
run_name = "make_model"

run_name = "make_model_cictt_full_canadair"

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))

In [25]:
df = pd.DataFrame(records)
df.head()

,left,right,match,match_confidence
0,COL make VAL CANADAIR COL model VAL CL215 6B11...,COL make VAL CESSNA COL model VAL T182 COL ser...,0,0.999963
1,COL make VAL CANADAIR COL model VAL CL600 2B16...,COL make VAL BOEING COL model VAL YL15 COL ser...,0,0.999959
2,COL make VAL CANADAIR COL model VAL CL600 2A12...,COL make VAL CESSNA COL model VAL 172 COL seri...,0,0.999971
3,COL make VAL CANADAIR COL model VAL CL600 2A12...,COL make VAL BOEING COL model VAL 737 COL seri...,0,0.999971
4,COL make VAL CANADAIR COL model VAL CL600 2A12...,COL make VAL CESSNA COL model VAL 172 COL seri...,0,0.999971


In [26]:


gold = pd.read_csv("../data/aircraft_er/" + run_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])
print(gold.head())

                                                left  \
0  COL make VAL CANADAIR COL model VAL CL215 6B11...   
1  COL make VAL CANADAIR COL model VAL CL600 2B16...   
2  COL make VAL CANADAIR COL model VAL CL600 2A12...   
3  COL make VAL CANADAIR COL model VAL CL600 2A12...   
4  COL make VAL CANADAIR COL model VAL CL600 2A12...   

                                               right  gold  
0  COL make VAL CESSNA COL model VAL T182 COL ser...     0  
1  COL make VAL BOEING COL model VAL YL15 COL ser...     0  
2  COL make VAL CESSNA COL model VAL 172 COL seri...     0  
3  COL make VAL BOEING COL model VAL 737 COL seri...     0  
4  COL make VAL CESSNA COL model VAL 172 COL seri...     0  


In [27]:
df["gold"] = gold["gold"]

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_true = df["gold"]
y_pred = df["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 1.0

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     17422
           1       1.00      1.00      1.00         8

    accuracy                           1.00     17430
   macro avg       1.00      1.00      1.00     17430
weighted avg       1.00      1.00      1.00     17430


Confusion matrix:
 [[17422     0]
 [    0     8]]


In [29]:
errors = df[df["gold"] != df["match"]]
print(errors[["left","right","gold","match","match_confidence"]].head(20))

Empty DataFrame
Columns: [left, right, gold, match, match_confidence]
Index: []


In [30]:
errors

,left,right,match,match_confidence,gold


In [31]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [32]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "gold": row["gold"],
        "predicted": row["match"],
        "confidence": row["match_confidence"]
    })

aligned = pd.DataFrame(parsed)

In [33]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)